Basically a notebook to test code interactively before writing it in full detail.

In [3]:
import numpy as np
import torch
import torchvision
import random
from torch.utils.data import Dataset, DataLoader, Sampler, BatchSampler


### Classwise Uniform Sampler

The goal is to write the sampling strategy of randomly pick among the K classes, and randomly pick P examples from these K classes in Pytorch.

In [4]:
class MyDataset(Dataset):
    def __init__(self, xs,ys):
        self.xs = xs
        self.ys = ys
        
        assert len(self.xs) == len(self.ys)
        
    def __len__(self):
        return len(self.xs)
    
    def __getitem__(self, i):
        return self.xs[i], self.ys[i]

In [95]:
mydataset = MyDataset(list(range(101,126)), [0,0,0,0,0,0, 1,1,1,1,2,2,2,2,2,2,3,3,3, 3,3,3, 4,4, 4])

In [96]:
start_indices = {x : 5 * x for x in range(5)} #index of first entry of that class
num_examples = {x : 5 for x in range(5)} #number of examples for each label

num_classes = 5

We want to handle a K * P batch. What about test time? Train/validation ? Does validation have the sample sample strategy? It should.

In [97]:
K = 3
P = 2

In [98]:
random.choices(list(range(5)), k=6)

[3, 1, 4, 0, 0, 3]

In [99]:
from collections import defaultdict
from loguru import logger

log = logger.info

In [106]:
class ClassUniformBatchSampler(Sampler):
    """Returns the classes in order, but with the entries of each class shuffled."""
    def __init__(self, dataset, P, K):
        """Returns batches of size P x K where P is the number of classes per batch and K is the number of samples per class."""
        self.num_classes = num_classes
        self.start_indices = start_indices
        self.num_examples = num_examples 
        self.dataset = dataset
        self.P = P
        self.K = K
        #log(f"P = {P}, K = {K}, len(dataset) = {len(dataset)}")
        
        
    def __iter__(self):
        """Provides a valid permutation of the indices for the dataset."""
        
        #Shuffle indices of each class in place.
        classwise_shuffled_indices = []
        k_at_time = defaultdict(list) #for each class we want the value to be [[k samples], [k samples], ...]
        #log(f"Before for loop in __iter__, num_classes = {num_classes}")
        for i in range(num_classes):
            class_batches = []
            start_idx = self.start_indices[i]
            stop_idx = start_idx + self.num_examples[i]
            #log(f"For class {i}, start_idx = {start_idx} and stop_idx = {stop_idx}")
            #want a random shuffle of the dataset from start_idx -> start_idx + num_examples
            shuffled_examples = random.sample(range(start_idx, stop_idx), stop_idx - start_idx)
            #log(f"For class {i}, shuffled_examples = {shuffled_examples}")
            split_batches = list(torch.split(torch.tensor(shuffled_examples), self.K))
            #log(f"For class {i}, split_batches start = {split_batches}")
            #last_batch = n % self.K
            
            #we want last batch to also have size K using random.choices
            split_batches[-1] = torch.tensor(random.choices(shuffled_examples[-(len(shuffled_examples) % self.K) :], k=self.K))
             #we want to pop() elements so the last batch should be at index 0
            #log(f"For class {i}, split_batches after resizing last element = {split_batches}")
            split_batches = split_batches[::-1]
            k_at_time[i] = [x.tolist() for x in split_batches] #each is a list of k indices.
            #log(f"For class {i}, k_at_time[i] = {k_at_time[i]}")
            classwise_shuffled_indices += shuffled_examples
            
        
        
        #We want to extract P classes at a time randomly and pop the k_at_a_time[idx] for each
        #If any class becomes empty, we want to remove it from our set of alive classes.
        #When we have fewer than K classes remaining, we just return what remains.
        
        alive_classes = set(range(num_classes))
        
        while len(alive_classes) >= self.P:
            #sample P classes
            #log(f"In while loop, alive_classes = {alive_classes}")
            selected_indices = random.sample(list(alive_classes), k=self.P)    
            x_batch  = []
            
            
            for idx in selected_indices:
                class_batch = k_at_time[idx].pop()
                if not k_at_time[idx]:
                    alive_classes.remove(idx)
                x_batch += class_batch
                #y_batch += [idx] * len(class_batch)
                
            
            random.shuffle(x_batch)
            #log(f"In while loop, x_batch = {x_batch}")
            yield x_batch
        
        #if there are fewer than P classes remaining, we will just cycle through each class and add a set of K elements in order
        #until we hit PxK elements. If we are done before we hit PxK elements, we ignore the batch. 
        log(f"alive_classes = {alive_classes}")
        curr_batch = []
        num_items_in_batch = 0
        while len(alive_classes) >= 2: #if there is only one alive class, no point having a triplet loss
            for idx in list(alive_classes):
                class_batch = k_at_time[idx].pop()
                if not k_at_time[idx]:
                    alive_classes.remove(idx)
                
                curr_batch += class_batch
                num_items_in_batch += self.K
            
                if num_items_in_batch == self.P * self.K:
                    random.shuffle(curr_batch)
                    yield curr_batch
                    curr_batch = []
                    num_items_in_batch = 0
                    
        #there are fewer than PxK elements remaining    
        
        return
    
    def __len__(self):
        return len(self.dataset)
    

In [112]:
mysampler = ClassUniformBatchSampler(dataset=mydataset, P=2, K=2) #each batch has 4 elements from 2 classes

In [114]:
i = 0
for i, indices in enumerate(mysampler):
    print(f"In batch {i}: \n indices = {indices} \n elements = {[mydataset[i] for i in indices]} \n\n")
    
print(f" i = {i}")

2022-11-16 15:08:26.080 | INFO     | __main__:__iter__:71 - alive_classes = {1}


In batch 0: 
 indices = [19, 12, 18, 10] 
 elements = [(120, 3), (113, 2), (119, 3), (111, 2)] 


In batch 1: 
 indices = [3, 13, 14, 1] 
 elements = [(104, 0), (114, 2), (115, 2), (102, 0)] 


In batch 2: 
 indices = [16, 24, 23, 17] 
 elements = [(117, 3), (125, 4), (124, 4), (118, 3)] 


In batch 3: 
 indices = [15, 11, 11, 15] 
 elements = [(116, 2), (112, 2), (112, 2), (116, 2)] 


In batch 4: 
 indices = [8, 6, 2, 0] 
 elements = [(109, 1), (107, 1), (103, 0), (101, 0)] 


In batch 5: 
 indices = [5, 21, 22, 9] 
 elements = [(106, 0), (122, 3), (123, 4), (110, 1)] 


In batch 6: 
 indices = [4, 20, 20, 4] 
 elements = [(105, 0), (121, 3), (121, 3), (105, 0)] 


 i = 6
